In [1]:
# !pip install gdown
# !pip install chromadb
# !pip install -U sentence-transformers

In [2]:
!gdown 1C0Qn6FUwNQtq5MowDF3Eheg5ilE4pPIB

Downloading...
From: https://drive.google.com/uc?id=1C0Qn6FUwNQtq5MowDF3Eheg5ilE4pPIB
To: /content/movies.csv
100% 494k/494k [00:00<00:00, 7.35MB/s]


In [3]:
import pandas as pd
df = pd.read_csv('/content/movies.csv')
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
df.dropna(inplace=True)

In [5]:
feature = df.title + " " + df['genres'].apply(lambda x: x.replace("|", " "))
feature = feature.str.lower()

In [6]:

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(feature.values).tolist()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
df.dtypes

,0
movieId,int64
title,object
genres,object


In [8]:
ids = df.movieId.astype(str).values.tolist()
metadata = df.to_dict('records')

In [9]:
import chromadb
chroma_client = chromadb.Client()

In [12]:
collection = chroma_client.create_collection("vector_db")

In [13]:
collection.add(ids = ids, documents=feature.values.tolist(), embeddings = embeddings, metadatas=metadata )

In [18]:
user_feature = input("What is your movie preference? :")

What is your movie preference? :science fiction, time travel 


In [19]:
user_feature

'science fiction, time travel '

In [20]:
# Semantic Search
recommendations = collection.query(query_texts=[user_feature],n_results=3)

In [21]:
print(recommendations['metadatas'])

[[{'genres': 'Drama|Romance|Sci-Fi', 'movieId': 70599, 'title': "Time Traveler's Wife, The (2009)"}, {'genres': 'Action|Adventure|Sci-Fi', 'movieId': 5171, 'title': 'Time Machine, The (2002)'}, {'genres': 'Comedy|Sci-Fi', 'movieId': 71106, 'title': 'Frequently Asked Questions About Time Travel (2009)'}]]
